In [3]:
import algorithmes.aco as aco
import random 
import numpy as np
import threading
import time

import utils.prog_lineraire as pl
import utils.generator_matrix as gm
import dataset
import json
import pandas as pd

ModuleNotFoundError: No module named 'algorithmes'

In [21]:
def generate_nodes(number,max_x=1080,max_y=720,seed=None):
    # create number of nodes with coordinates of x,y  with max value of 600
    random.seed(seed)
    nodes = []
    for i in range(number):
        nodes.append([random.randint(0.1*max_y,max_x-0.1*max_y),random.randint(0.1*max_y,max_y-0.1*max_y)])
    return nodes

def distance(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def coordinats_to_matrice(nodes):
    # create a matrice with the nodes coordinates
    matrice = np.zeros((len(nodes),len(nodes)))
    for x in range(len(nodes)):
        for y in range(len(nodes)):
            matrice[x][y] = distance(nodes[x][0],nodes[x][1],nodes[y][0],nodes[y][1])
    return matrice

In [22]:
#
#nbr_iter=100
#nbr_ant=10
#a_pheromone = 1
#b_heuristic = 1
#p_evaporate = 0.1
#size = 10
#seed= 10
#
#matrice_pondere = coordinats_to_matrice(generate_nodes(10,seed=seed))
#
#pheromone,best_path,best_score,time_record = aco.aco_algorithm(matrice_pondere,nbr_iter,nbr_ant,a_pheromone,b_heuristic,p_evaporate,seed)
#
#

In [23]:

parameters = {
    "nbr_ant":[100],#np.arange(25,500,25)
    "a":np.arange(0,4.25,.25),
    "b":np.arange(1,4.25,.25),
    "p":np.arange(0,1.05,.05),
    "nb_iter_max":[1000],
    "time_max":[60],
    "nbr_random_search":[10],
    "nbr_loops":[3],
}

constraint = {
    "enabled":False,
    "start":4,
    "city":18,
    "inferior_dist":1000,
    "superior_dist":2000,
}


In [24]:


#matrice= dataset.import_dataset("dataset/small_dataset")
#villes = dataset.import_dataset("dataset/small_cities")
#
##datasets = generate_datasets(nbr_dataset,nbr_sommet,seed=None)
#datasets = [matrice]
#borne_inf = pl.get_borne_inf(matrice,villes)
#print(borne_inf)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/small_dataset.pickle'

# Optimisation
Comment evaluer mes algorithmes ?
Comment faire pour comparer les algorithmes ?

Déterminer la borne pour chacun des dataset

Tester une combiaison de paramètres sur chaque dataset pour obtenir une moyenne et l'ecart type:


1. Prendre une combinaison de paramètres
2. Tester les paramètres sur chaque dataset 10 fois.
   1. Récupérer la moyenne et l'écart type des 10 essais sur chaque dataset.
1. Evaluer l'effcatité d'une combinaison.
   






In [ ]:
def random_parameters(parameters):
    """Return random parameters in parameters

    Args:
        parameters (array): array of parameters

    Returns:
        dict: return dict of random parameters
    """
    return {key:random.choice(parameters[key]) for key in parameters}


In [ ]:
#df =   pd.DataFrame(columns=["nbr_ant","a","b","p","nb_iter_max","time_max","nbr_random_search","nbr_loops","mean_score","std_score"])
#df.to_json("dataset/results_rdm_search_cv.json")

In [ ]:
def GridSearchCV(parameters,datasets,constraint):


    for rdm_search in range(parameters["nbr_random_search"][0]):
        
        parameter = random_parameters(parameters)

        print("Random search : {}/{}".format(rdm_search+1,parameters["nbr_random_search"][0]))
        print("Parameters : {}".format(parameter))

        mean_scores = []
        standard_deviations = []


        for dataset in datasets:
            #
            

            mean_score = 0
            aco_result_list = []

            thrd_list = []
            start = time.time()

            for loop in range(parameters["nbr_loops"][0]):
                thread = threading.Thread(target=aco.aco_algorithm, args=(dataset,parameter,aco_result_list,constraint,))
                thrd_list.append(thread)
                thread.start()
                
                
            for thread in thrd_list:  # iterates over the threads
                #difference de temps avec start
                
                thread.join()     
                #print("Temps d'execution: " + str(time.time() - start))
            


            # pourcentage de différence entre la programation linéaire et l'aco

            mean_score = np.mean([borne_inf*100/(aco.path_distance(aco_result_list[i][1],matrice)) for i in range(len(aco_result_list))])
            standard_deviation = np.std([borne_inf*100/(aco.path_distance(aco_result_list[i][1],matrice)) for i in range(len(aco_result_list))])

            df = pd.read_json("dataset/results_rdm_search_cv.json")
            row = {"nbr_ant":parameter["nbr_ant"],"a":parameter["a"],"b":parameter["b"],"p":parameter["p"],"nb_iter_max":parameter["nb_iter_max"],"time_max":parameter["time_max"],"nbr_random_search":parameter["nbr_random_search"],"nbr_loops":parameter["nbr_loops"],"score":mean_score,"std_score":standard_deviation}
            df = df.append(row,ignore_index=True)
            df.to_json("dataset/results_rdm_search_cv.json")

        


In [ ]:

GridSearchCV(parameters,datasets,constraint)

Random search : 1/10
Parameters : {'nbr_ant': 100, 'a': 1.5, 'b': 1.0, 'p': 0.65, 'nb_iter_max': 1000, 'time_max': 60, 'nbr_random_search': 10, 'nbr_loops': 5}
iteration :  0 / 1000  score :  8341.394341576424
path :  [0, 17, 5, 23, 21, 16, 13, 10, 6, 8, 19, 11, 1, 4, 18, 2, 20, 3, 12, 7, 14, 24, 9, 15, 22, 0]
iteration :  0 / 1000  score :  15102.251400440882
path :  [0, 18, 23, 5, 12, 1, 4, 3, 13, 17, 19, 11, 20, 22, 9, 15, 2, 8, 14, 24, 21, 10, 6, 16, 7, 0]
iteration :  0 / 1000  score :  16602.548418837232
path :  [0, 19, 16, 15, 6, 13, 8, 4, 1, 17, 5, 23, 2, 9, 14, 12, 22, 18, 7, 24, 21, 20, 11, 10, 3, 0]
iteration :  0 / 1000  score :  17659.392416065275
path :  [0, 4, 1, 15, 21, 9, 24, 12, 16, 18, 14, 7, 23, 5, 10, 3, 13, 8, 20, 11, 2, 22, 19, 17, 6, 0]
iteration :  0 / 1000  score :  16166.319268079555
path :  [0, 22, 13, 18, 14, 10, 21, 16, 4, 1, 3, 20, 12, 17, 19, 11, 5, 23, 7, 6, 8, 24, 9, 15, 2, 0]
iteration :  1 / 1000  score :  15102.251400440882
path :  [0, 18, 23, 5, 12

/tmp/ipykernel_76808/322798362.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


iteration :  0 / 1000  score :  16845.126494924243
path :  [0, 15, 4, 12, 20, 17, 19, 16, 13, 3, 18, 22, 2, 21, 5, 10, 7, 14, 24, 23, 9, 1, 11, 6, 8, 0]
iteration :  0 / 1000  score :  17728.260358670097
path :  [0, 19, 18, 22, 12, 17, 5, 13, 8, 7, 14, 23, 15, 4, 1, 20, 11, 24, 21, 10, 16, 3, 6, 9, 2, 0]
iteration :  0 / 1000  score :  18834.45956873558
path :  [0, 3, 6, 2, 18, 14, 7, 22, 11, 13, 21, 17, 20, 5, 1, 23, 16, 10, 19, 8, 4, 24, 12, 9, 15, 0]
iteration :  1 / 1000  score :  17728.260358670097
path :  [0, 19, 18, 22, 12, 17, 5, 13, 8, 7, 14, 23, 15, 4, 1, 20, 11, 24, 21, 10, 16, 3, 6, 9, 2, 0]
iteration :  0 / 1000  score :  18194.92771035996
path :  [0, 12, 17, 1, 4, 15, 9, 23, 7, 14, 5, 13, 18, 22, 21, 2, 10, 16, 19, 11, 20, 8, 6, 3, 24, 0]
iteration :  1 / 1000  score :  16845.126494924243
path :  [0, 15, 4, 12, 20, 17, 19, 16, 13, 3, 18, 22, 2, 21, 5, 10, 7, 14, 24, 23, 9, 1, 11, 6, 8, 0]
iteration :  0 / 1000  score :  16499.508325617448
path :  [0, 11, 20, 22, 19, 14, 7

/tmp/ipykernel_76808/322798362.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


Random search : 3/10
Parameters : {'nbr_ant': 100, 'a': 3.25, 'b': 2.75, 'p': 0.6000000000000001, 'nb_iter_max': 1000, 'time_max': 60, 'nbr_random_search': 10, 'nbr_loops': 5}
iteration :  0 / 1000  score :  14745.549872264337
path :  [0, 15, 24, 9, 17, 5, 11, 20, 12, 22, 18, 14, 19, 16, 10, 23, 7, 8, 2, 6, 13, 1, 4, 3, 21, 0]
iteration :  0 / 1000  score :  14816.62970129858
path :  [0, 22, 18, 7, 14, 3, 20, 11, 19, 17, 5, 23, 9, 15, 4, 10, 12, 1, 16, 13, 6, 8, 21, 2, 24, 0]
iteration :  0 / 1000  score :  7319.324882862368
path :  [0, 15, 2, 6, 12, 4, 1, 16, 19, 17, 18, 14, 7, 23, 5, 13, 3, 20, 11, 10, 22, 24, 21, 8, 9, 0]
iteration :  1 / 1000  score :  14745.549872264337
path :  [0, 15, 24, 9, 17, 5, 11, 20, 12, 22, 18, 14, 19, 16, 10, 23, 7, 8, 2, 6, 13, 1, 4, 3, 21, 0]
iteration :  0 / 1000  score :  12717.269339112005
path :  [0, 22, 18, 19, 17, 5, 23, 13, 3, 20, 11, 12, 1, 4, 6, 15, 24, 14, 7, 8, 9, 2, 10, 16, 21, 0]
iteration :  0 / 1000  score :  16288.203521607316
path :  [0

...

/tmp/ipykernel_76808/322798362.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


iteration :  0 / 1000  score :  12891.179473176957
path :  [0, 15, 24, 14, 7, 16, 5, 6, 17, 23, 18, 19, 1, 4, 12, 22, 10, 9, 8, 2, 3, 20, 11, 13, 21, 0]
iteration :  0 / 1000  score :  6767.537522833478
path :  [0, 22, 19, 6, 5, 17, 12, 4, 1, 7, 14, 24, 15, 18, 16, 13, 8, 2, 9, 10, 23, 21, 3, 20, 11, 0]
iteration :  0 / 1000  score :  12038.221263556548
path :  [0, 22, 23, 18, 7, 14, 24, 15, 1, 4, 12, 17, 5, 16, 3, 11, 20, 19, 10, 6, 13, 8, 2, 9, 21, 0]
iteration :  1 / 1000  score :  12891.179473176957
path :  [0, 15, 24, 14, 7, 16, 5, 6, 17, 23, 18, 19, 1, 4, 12, 22, 10, 9, 8, 2, 3, 20, 11, 13, 21, 0]
iteration :  0 / 1000  score :  11760.290200687972
path :  [0, 22, 7, 14, 24, 15, 4, 5, 16, 17, 23, 12, 6, 13, 10, 1, 19, 18, 9, 8, 2, 21, 20, 11, 3, 0]
iteration :  1 / 1000  score :  6767.537522833478
path :  [0, 22, 19, 6, 5, 17, 12, 4, 1, 7, 14, 24, 15, 18, 16, 13, 8, 2, 9, 10, 23, 21, 3, 20, 11, 0]
iteration :  0 / 1000  score :  11539.678196632547
path :  [0, 15, 10, 23, 12, 4, 3,

/tmp/ipykernel_76808/322798362.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


iteration :  0 / 1000  score :  13272.257506650001
path :  [0, 22, 18, 7, 14, 2, 8, 9, 17, 19, 5, 23, 10, 4, 1, 12, 6, 3, 11, 20, 16, 21, 15, 24, 13, 0]
iteration :  0 / 1000  score :  12946.477982438171
path :  [0, 4, 12, 11, 20, 22, 17, 19, 13, 6, 5, 23, 18, 2, 9, 8, 14, 24, 15, 7, 1, 10, 16, 21, 3, 0]
iteration :  0 / 1000  score :  11867.71196326253
path :  [0, 22, 18, 7, 14, 24, 15, 10, 17, 5, 19, 1, 4, 12, 6, 3, 20, 11, 16, 13, 8, 9, 2, 23, 21, 0]
iteration :  0 / 1000  score :  12789.879908932959
path :  [0, 22, 7, 18, 19, 5, 6, 2, 9, 15, 24, 14, 23, 17, 12, 4, 1, 20, 11, 3, 16, 10, 13, 8, 21, 0]
iteration :  1 / 1000  score :  13272.257506650001
path :  [0, 22, 18, 7, 14, 2, 8, 9, 17, 19, 5, 23, 10, 4, 1, 12, 6, 3, 11, 20, 16, 21, 15, 24, 13, 0]
iteration :  1 / 1000  score :  12946.477982438171
path :  [0, 4, 12, 11, 20, 22, 17, 19, 13, 6, 5, 23, 18, 2, 9, 8, 14, 24, 15, 7, 1, 10, 16, 21, 3, 0]
iteration :  0 / 1000  score :  13954.334819587211
path :  [0, 22, 18, 7, 14, 24, 1

/tmp/ipykernel_76808/322798362.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


In [2]:
df = pd.read_json("dataset/results_rdm_search_cv.json").sort_values(by="score",ascending=False)
print(df)

NameError: name 'pd' is not defined